<a href="https://colab.research.google.com/github/mengwangk/dl-projects/blob/master/04_09_auto_ml_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automated ML - Generate Data

In [ ]:
COLAB = True

DATASET_NAME = '4D.zip'

FEATURE_DATASET_PREFIX = 'feature_matrix_snapshot'

ORIG_FEATURE_DATASET_PREFIX = 'feature_matrix_d2_v3'

TARGET_MONTH_YEAR = pd.datetime(2019,12,1)

In [2]:
if COLAB:
  !pip install -U featuretools
  !rm -rf dl-projects
  !git clone https://github.com/mengwangk/dl-projects
  !cd dl-projects && ls
else:
  !pip install -U featuretools==0.13.2

     |████████████████████████████████| 245kB 9.4MB/s eta 0:00:01
     |████████████████████████████████| 61kB 8.3MB/s  eta 0:00:01
  Created wheel for locket: filename=locket-0.2.0-cp36-none-any.whl size=4040 sha256=2bf5c763522dc4658110f3b31625b7a23d7a73d3a0384bbc6478879958e62608
  Stored in directory: /root/.cache/pip/wheels/26/1e/e8/4fa236ec931b1a0cdd61578e20d4934d7bf188858723b84698
Successfully built locket
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1
  Found existing installation: featuretools 0.4.1
    Uninstalling featuretools-0.4.1:
      Successfully uninstalled featuretools-0.4.1


Cloning into 'dl-projects'...
remote: Enumerating objects: 159, done.
remote: Counting objects: 100% (159/159), done.
remote: Compressing objects: 100% (146/146), done.
remote: Total 1784 (delta 100), reused 31 (delta 13), pack-reused 1625
Receiving objects: 100% (1784/1784), 76.71 MiB | 27.93 MiB/s, done.
Resolving deltas: 100% (1095/1095), done.
01_01_tensor.ipynb	       04_04_auto_ml_2.ipynb  07_algebra.ipynb
01_02_correlation.ipynb        04_04_auto_ml_3.ipynb  08_probability.ipynb
01_03_preprocessing.ipynb      04_04_auto_ml_4.ipynb  09_evaluation.ipynb
01_04_numpy.ipynb	       04_05_auto_ml_0.ipynb  50_RL.ipynb
01_05_parallelization.ipynb    04_05_auto_ml_1.ipynb  999_Links.ipynb
01_06_pandas..ipynb	       04_05_auto_ml_2.ipynb  archive
01_07_python_0.ipynb	       04_06_auto_ml_1.ipynb  d01_download.ipynb
01_07_python_1.ipynb	       04_06_auto_ml_2.ipynb  d02_0_features.ipynb
01_07_python_2.ipynb	       04_06_auto_ml_3.ipynb  d02_1_features.ipynb
01_08_sklearn_1.ipynb	       04_0

In [ ]:
if COLAB:
  !cp dl-projects/utils* .
  !cp dl-projects/preprocess* .

In [ ]:
%reload_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as ss
import math 
import matplotlib
import re

from scipy import stats
from collections import Counter
from pathlib import Path

plt.style.use('fivethirtyeight')

sns.set(style="ticks")

# Automated feature engineering
import featuretools as ft

# Machine learning
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, roc_curve
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from dateutil.relativedelta import relativedelta

from IPython.display import display

from utils import *
from preprocess import *

# The Answer to the Ultimate Question of Life, the Universe, and Everything.
np.random.seed(42)

In [7]:
%aimport

Modules to reload:
all-except-skipped

Modules to skip:



## Preparation

In [8]:
if COLAB:
  from google.colab import drive
  drive.mount('/content/gdrive')
  GDRIVE_DATASET_FOLDER = Path('gdrive/My Drive/datasets/')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
if COLAB:
  DATASET_PATH = GDRIVE_DATASET_FOLDER
  ORIGIN_DATASET_PATH = Path('dl-projects/datasets')
else:
  DATASET_PATH = Path("../datasets")
  ORIGIN_DATASET_PATH = Path('datasets')

DATASET = DATASET_PATH/f"{FEATURE_DATASET_PREFIX}.ft"
ORIGIN_DATASET = ORIGIN_DATASET_PATH/DATASET_NAME

In [ ]:
data = format_tabular(ORIGIN_DATASET)

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106858 entries, 0 to 106857
Data columns (total 4 columns):
DrawNo       106858 non-null int64
DrawDate     106858 non-null datetime64[ns]
PrizeType    106858 non-null object
LuckyNo      106858 non-null int64
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 3.3+ MB


In [12]:
data.tail(24)

,DrawNo,DrawDate,PrizeType,LuckyNo
106834,505219,2019-12-29,SpecialNo9,2528
106835,505320,2020-01-01,1stPrizeNo,875
106836,505320,2020-01-01,2ndPrizeNo,6307
106837,505320,2020-01-01,3rdPrizeNo,8552
106838,505320,2020-01-01,ConsolationNo1,2114
106839,505320,2020-01-01,ConsolationNo10,4861
106840,505320,2020-01-01,ConsolationNo2,508
106841,505320,2020-01-01,ConsolationNo3,2086
106842,505320,2020-01-01,ConsolationNo4,3296
106843,505320,2020-01-01,ConsolationNo5,413


In [ ]:
data['NumberId'] = data['LuckyNo']

In [14]:
data.tail(10)

,DrawNo,DrawDate,PrizeType,LuckyNo,NumberId
106848,505320,2020-01-01,SpecialNo1,2404,2404
106849,505320,2020-01-01,SpecialNo10,7382,7382
106850,505320,2020-01-01,SpecialNo2,2220,2220
106851,505320,2020-01-01,SpecialNo3,3573,3573
106852,505320,2020-01-01,SpecialNo4,4524,4524
106853,505320,2020-01-01,SpecialNo5,6856,6856
106854,505320,2020-01-01,SpecialNo6,8107,8107
106855,505320,2020-01-01,SpecialNo7,1026,1026
106856,505320,2020-01-01,SpecialNo8,3960,3960
106857,505320,2020-01-01,SpecialNo9,5472,5472


## Data Cleansing

In [ ]:
def change_prize_type(x):
  return "Prize"
  # if "Special" in x:
  #   return 'Prize'
  # elif "Consolation" in x:
  #   return 'Prize'
  # else:
  #   return x

def increment_draw_date(x):
  x = x + relativedelta(minutes=1)
  return x

data['PrizeType'] = data['PrizeType'].apply(change_prize_type)

# Add 1 minutes to draw date to avoid overlapping with the cut off date
data['DrawDate'] = data['DrawDate'].apply(increment_draw_date)

In [ ]:
data.tail(23)

In [ ]:
data.describe().round(2)

In [ ]:
plt.figure(figsize=(20,6))
sns.boxplot(x='NumberId', y='PrizeType',data=data)
plt.xticks(rotation=90)
plt.title('Draw')

# print(data[data['NumberId']==1760])

## Making Labels

In [ ]:
data['TotalStrike'] = 1
data.head(10)

In [ ]:
def make_cutoffs(start_date, end_date, threshold=0):
    # Find numbers exist before start date
    number_pool = data[data['DrawDate'] < start_date]['NumberId'].unique()
    tmp = pd.DataFrame({'NumberId': number_pool})
   
    # For numbers in the number pool, find their strike count between the start and end dates
    strike_counts = data[(data['NumberId'].isin(number_pool)) & 
        (data['DrawDate'] >= start_date) & 
        (data['DrawDate']< end_date)
    ].groupby('NumberId')['TotalStrike'].count().reset_index()
    
    number_of_draws = data[
                 (data['DrawDate'] >= start_date) & 
                 (data['DrawDate']< end_date)]['DrawDate'].nunique()
    # display(strike_counts)
    # print(number_of_draws)
    
    # Merge with all the number ids to record all customers who existed before start date
    strike_counts = strike_counts.merge(tmp, on='NumberId', how='right')
    
    # Set the total for any numbers who did not strike in the timeframe equal to 0
    strike_counts['TotalStrike'] = strike_counts['TotalStrike'].fillna(0)
    
    # Label is based on the threshold
    strike_counts['Label'] = (strike_counts['TotalStrike'] > threshold).astype(int)
        
    # The cutoff time is the start date
    strike_counts['cutoff_time'] = pd.to_datetime(start_date)
    strike_counts = strike_counts[['NumberId', 'cutoff_time', 'TotalStrike', 'Label']]
    
    #display(strike_counts[strike_counts['Label']==1].nunique())
    #display(strike_counts.sort_values(by='TotalStrike', ascending=False))
   
    return number_of_draws, strike_counts

In [ ]:
# For testing purpose
data = data[data['DrawDate'] < TARGET_MONTH_YEAR]

In [ ]:
data.tail(10)

In [ ]:
end_month_year = TARGET_MONTH_YEAR +  relativedelta(months=1)
end_month_year

In [ ]:
number_of_draws, target_month_year = make_cutoffs(TARGET_MONTH_YEAR, end_month_year)
print(number_of_draws, len(target_month_year))
target_month_year[(target_month_year['Label']==1) & (target_month_year['TotalStrike']==2)].sort_values(by='TotalStrike', ascending=False).head()

In [ ]:
data[data['NumberId'].isin([135,2513,5030])]

In [ ]:
target_month_year['Label'].value_counts().plot.bar()
plt.title('Label Distribution for Target Month Year')

In [ ]:
CUT_OFF_YEAR = TARGET_MONTH_YEAR

In [ ]:
## Loop through each month starting from CUT_OFF_YEAR

# print(data['DrawDate'].max())
max_year_month = TARGET_MONTH_YEAR  + relativedelta(day=31)
print(f"Max month year: {max_year_month}")

start_year_month = CUT_OFF_YEAR

months_data = []
total_draws = 0
while start_year_month < max_year_month:
    start_date = start_year_month 
    end_date = start_date + relativedelta(months=1)
    start_year_month = start_year_month + relativedelta(months=1)
    print(f"Labels from {start_date} to {end_date}")
    draw_count, month_data = make_cutoffs(start_date, end_date)
    total_draws = total_draws + draw_count
    months_data.append(month_data)

print(f"Total draws: {total_draws}")
print(f"Total draws: {data[(data['DrawDate'] >= CUT_OFF_YEAR) & (data['DrawDate'] <= max_year_month)]['DrawDate'].nunique()}")
print(f"Total months:{len(months_data)}")
print(f"Total records count: {sum([len(l) for l in months_data])}")
print([len(l) for l in months_data])

In [ ]:
labels = pd.concat(months_data)
labels.to_csv(DATASET_PATH/'labels.csv')
labels.describe()

In [ ]:
labels.NumberId.nunique(), labels.head(10)

In [ ]:
labels.tail(10)

## Automated Feature Engineering

In [ ]:
es = ft.EntitySet(id="Lotto Results")

# Add the entire data table as an entity
es.entity_from_dataframe("Results",
                         dataframe=data,
                         index="results_index",
                         time_index = 'DrawDate')

es['Results']

In [ ]:
es.normalize_entity(new_entity_id="Numbers",
                    base_entity_id="Results",
                    index="NumberId",
                    )

In [ ]:
es

In [ ]:
es['Numbers'].df.head(24)

In [ ]:
es['Results'].df.head(24)

In [ ]:
len(es['Results'].df)

## Deep Feature Synthesis

In [ ]:
# feature_matrix, feature_names = ft.dfs(entityset=es, target_entity='Numbers',
#                                         cutoff_time = labels, verbose = 2,
#                                         cutoff_time_in_index = True,
#                                         chunk_size = len(labels), n_jobs = 1,
#                                         max_depth = 1)
print(ft.__version__)
ft.list_primitives().T

In [ ]:
# default_agg_primitives =  ['std', 'max', 'min', 'mode', 'mean', 'skew', 'last', 'avg_time_between', 
#                            'count', 'sum', 'trend', 'percent_true', 'num_unique', 'time_since_last', 
#                            'time_since_first']
# default_trans_primitives =  ['cum_sum', 'cum_mean', 'month', 'day', 'percentile', 
#                               'time_since', 'time_since_previous', 'cum_min', 'cum_max']

default_agg_primitives =  ['std', 'max', 'min', 'mode', 'mean', 'skew', 'last', 'avg_time_between', 
                           'count', 'sum', 'trend', 'percent_true', 'num_unique']
default_trans_primitives =  ['cum_sum', 'cum_mean', 'month', 'day', 'percentile', 
                            'and', 'time_since', 'time_since_previous']

feature_matrix, feature_names = ft.dfs(entityset=es, target_entity='Numbers',
                                        agg_primitives = default_agg_primitives ,
                                        trans_primitives = default_trans_primitives,
                                        cutoff_time = labels, verbose = 1,
                                        cutoff_time_in_index = True,
                                        chunk_size = len(labels), n_jobs = 1,
                                        #chunk_size = 10000, n_jobs = -1,
                                        max_depth = 2)

In [ ]:
len(feature_matrix.columns), feature_matrix.columns

In [ ]:
len(feature_matrix)

In [ ]:
feature_matrix.head().T

In [ ]:
feature_matrix.shape

In [ ]:
feature_matrix.head(2)

In [ ]:
#feature_matrix[(feature_matrix['NumberId']==0) & (feature_matrix['Label']==1)].head(10)

## Correlations

In [ ]:
# Save the original feature matrix
origin_feature_matrix = feature_matrix

feature_matrix = pd.get_dummies(feature_matrix).reset_index()
feature_matrix.shape

In [ ]:
feature_matrix.head(10)

In [ ]:
corrs = feature_matrix.corr().sort_values('TotalStrike')
corrs['TotalStrike'].head()

In [ ]:
corrs['TotalStrike'].dropna().tail(60)

In [ ]:
feature_matrix['month'] = feature_matrix['time'].dt.month
feature_matrix['year'] = feature_matrix['time'].dt.year

In [ ]:
feature_matrix.info()

In [ ]:
feature_matrix.head()

In [ ]:
feature_matrix.tail()

In [ ]:
display(feature_matrix.describe())

## Save feature matrix

In [ ]:
# https://towardsdatascience.com/downloading-datasets-into-google-drive-via-google-colab-bcb1b30b0166
if COLAB:
    #!ls /content/gdrive/My\ Drive/
    feature_matrix.to_feather(DATASET)
    origin_feature_matrix.to_pickle(DATASET_PATH/f"{FEATURE_DATASET_PREFIX}_orig.pkl")

In [ ]:
if COLAB:
  !cd gdrive/"My Drive"/datasets/ && ls -l --block-size=M

### Validation

In [ ]:
feature_matrix = pd.read_feather(DATASET)

In [17]:
feature_matrix.shape

(10000, 217)

In [18]:
feature_matrix.describe().round(2)

,NumberId,STD(Results.LuckyNo),STD(Results.DrawNo),STD(Results.TotalStrike),MAX(Results.LuckyNo),MAX(Results.DrawNo),MAX(Results.TotalStrike),MIN(Results.LuckyNo),MIN(Results.DrawNo),MIN(Results.TotalStrike),MEAN(Results.LuckyNo),MEAN(Results.DrawNo),MEAN(Results.TotalStrike),SKEW(Results.LuckyNo),SKEW(Results.DrawNo),SKEW(Results.TotalStrike),LAST(Results.LuckyNo),LAST(Results.DrawNo),LAST(Results.results_index),LAST(Results.TotalStrike),AVG_TIME_BETWEEN(Results.DrawDate),COUNT(Results),SUM(Results.LuckyNo),SUM(Results.DrawNo),SUM(Results.TotalStrike),"TREND(Results.LuckyNo, DrawDate)","TREND(Results.DrawNo, DrawDate)","TREND(Results.TotalStrike, DrawDate)",NUM_UNIQUE(Results.PrizeType),MONTH(first_Results_time),DAY(first_Results_time),TIME_SINCE(first_Results_time),TIME_SINCE_PREVIOUS(first_Results_time),STD(Results.PERCENTILE(TotalStrike)),STD(Results.PERCENTILE(DrawNo)),STD(Results.CUM_SUM(TotalStrike)),STD(Results.CUM_MEAN(TotalStrike)),STD(Results.CUM_MEAN(LuckyNo)),STD(Results.PERCENTILE(LuckyNo)),STD(Results.CUM_SUM(DrawNo)),...,CUM_MEAN(SKEW(Results.DrawNo)),CUM_MEAN(LAST(Results.LuckyNo)),CUM_MEAN(SUM(Results.LuckyNo)),CUM_MEAN(MAX(Results.DrawNo)),CUM_MEAN(LAST(Results.DrawNo)),MONTH(LAST(Results.DrawDate)),DAY(LAST(Results.DrawDate)),PERCENTILE(MEAN(Results.DrawNo)),PERCENTILE(MIN(Results.DrawNo)),"PERCENTILE(TREND(Results.DrawNo, DrawDate))",PERCENTILE(MAX(Results.LuckyNo)),"PERCENTILE(TREND(Results.TotalStrike, DrawDate))",PERCENTILE(MIN(Results.LuckyNo)),PERCENTILE(SUM(Results.DrawNo)),PERCENTILE(LAST(Results.TotalStrike)),PERCENTILE(MIN(Results.TotalStrike)),PERCENTILE(STD(Results.LuckyNo)),PERCENTILE(MEAN(Results.LuckyNo)),PERCENTILE(SKEW(Results.LuckyNo)),PERCENTILE(STD(Results.TotalStrike)),PERCENTILE(COUNT(Results)),"PERCENTILE(TREND(Results.LuckyNo, DrawDate))",PERCENTILE(STD(Results.DrawNo)),PERCENTILE(AVG_TIME_BETWEEN(Results.DrawDate)),PERCENTILE(MAX(Results.TotalStrike)),PERCENTILE(MEAN(Results.TotalStrike)),PERCENTILE(SUM(Results.TotalStrike)),PERCENTILE(NUM_UNIQUE(Results.PrizeType)),PERCENTILE(SKEW(Results.TotalStrike)),PERCENTILE(SKEW(Results.DrawNo)),PERCENTILE(LAST(Results.LuckyNo)),PERCENTILE(SUM(Results.LuckyNo)),PERCENTILE(MAX(Results.DrawNo)),PERCENTILE(LAST(Results.DrawNo)),TotalStrike,Label,MODE(Results.PrizeType)_Prize,LAST(Results.PrizeType)_Prize,month,year
count,10000.00,10000.0,10000.00,10000.0,10000.00,10000.00,10000.0,10000.00,10000.00,10000.0,10000.00,10000.00,10000.0,9991.0,9991.00,9991.0,10000.00,10000.00,10000.00,10000.0,1.000000e+04,10000.00,10000.00,10000.00,10000.00,9991.0,9991.00,9991.0,10000.0,10000.00,10000.00,1.000000e+04,9999.00,10000.0,10000.00,10000.00,10000.0,10000.00,10000.0,1.000000e+04,...,9991.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,9991.00,10000.00,9991.0,10000.00,10000.00,10000.0,10000.0,10000.0,10000.00,9991.0,10000.0,10000.00,9991.0,10000.00,10000.00,10000.0,10000.0,10000.00,10000.0,9991.0,9991.00,10000.00,10000.00,10000.00,10000.00,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0
mean,4999.50,0.0,131752.36,0.0,4999.50,460862.61,1.0,4999.50,83988.68,1.0,4999.50,272245.28,1.0,0.0,0.00,0.0,4999.50,460862.61,96621.59,1.0,8.171873e+07,10.65,52983.24,2900347.17,10.65,0.0,0.03,0.0,1.0,6.67,15.66,7.834829e+08,72764.72,0.0,0.28,30300.18,0.0,10.96,0.0,8.408713e+09,...,0.00,4997.24,56196.04,460482.34,460482.34,6.77,15.88,0.50,0.50,0.50,0.50,0.5,0.50,0.50,0.5,0.5,0.5,0.50,0.5,0.5,0.50,0.5,0.50,0.50,0.5,0.5,0.50,0.5,0.5,0.50,0.50,0.50,0.50,0.50,0.0,0.0,1.0,1.0,12.0,2019.0
std,2886.90,0.0,23271.77,0.0,2886.90,42603.42,0.0,2886.90,43002.59,0.0,2886.90,43398.78,0.0,0.0,0.59,0.0,2886.90,42603.42,9798.22,0.0,3.367741e+07,3.28,35730.66,995886.40,3.28,0.0,0.00,0.0,0.0,3.37,8.81,8.525254e+07,689802.39,0.0,0.05,5352.38,0.0,39.11,0.0,1.692155e+09,...,0.01,136.49,2123.89,1128.85,1128.85,3.38,9.11,0.29,0.29,0.29,0.29,0.0,0.29,0.29,0.0,0.0,0.0,0.29,0.0,0.0,0.29,0.0,0.29,0.29,0.0,0.0,0.29,0.0,0.0,0.29,0.29,0.29,0.29,0.29,0.0,0.0,0.0,0.0,0.0,0.0
min,0.00

In [ ]:
compare_with = pd.read_feather(DATASET_PATH/f"{ORIG_FEATURE_DATASET_PREFIX}.ft")

In [20]:
compare_with.tail(10)

,NumberId,time,STD(Results.DrawNo),STD(Results.TotalStrike),STD(Results.LuckyNo),MAX(Results.DrawNo),MAX(Results.TotalStrike),MAX(Results.LuckyNo),MIN(Results.DrawNo),MIN(Results.TotalStrike),MIN(Results.LuckyNo),MEAN(Results.DrawNo),MEAN(Results.TotalStrike),MEAN(Results.LuckyNo),SKEW(Results.DrawNo),SKEW(Results.TotalStrike),SKEW(Results.LuckyNo),LAST(Results.results_index),LAST(Results.DrawNo),LAST(Results.TotalStrike),LAST(Results.LuckyNo),AVG_TIME_BETWEEN(Results.DrawDate),COUNT(Results),SUM(Results.DrawNo),SUM(Results.TotalStrike),SUM(Results.LuckyNo),"TREND(Results.TotalStrike, DrawDate)","TREND(Results.LuckyNo, DrawDate)","TREND(Results.DrawNo, DrawDate)",NUM_UNIQUE(Results.PrizeType),MONTH(first_Results_time),DAY(first_Results_time),TIME_SINCE(first_Results_time),TIME_SINCE_PREVIOUS(first_Results_time),STD(Results.TIME_SINCE(DrawDate)),STD(Results.CUM_MEAN(DrawNo)),STD(Results.CUM_SUM(LuckyNo)),STD(Results.PERCENTILE(DrawNo)),STD(Results.TIME_SINCE_PREVIOUS(DrawDate)),STD(Results.CUM_SUM(TotalStrike)),...,CUM_MEAN(SKEW(Results.TotalStrike)),CUM_MEAN(SUM(Results.DrawNo)),"CUM_MEAN(TREND(Results.DrawNo, DrawDate))",CUM_MEAN(LAST(Results.LuckyNo)),CUM_MEAN(MAX(Results.LuckyNo)),MONTH(LAST(Results.DrawDate)),DAY(LAST(Results.DrawDate)),PERCENTILE(SKEW(Results.LuckyNo)),PERCENTILE(MIN(Results.DrawNo)),PERCENTILE(MEAN(Results.DrawNo)),PERCENTILE(STD(Results.LuckyNo)),PERCENTILE(LAST(Results.DrawNo)),PERCENTILE(MAX(Results.TotalStrike)),"PERCENTILE(TREND(Results.TotalStrike, DrawDate))",PERCENTILE(MEAN(Results.TotalStrike)),PERCENTILE(MIN(Results.TotalStrike)),PERCENTILE(AVG_TIME_BETWEEN(Results.DrawDate)),PERCENTILE(MEAN(Results.LuckyNo)),PERCENTILE(MAX(Results.DrawNo)),PERCENTILE(COUNT(Results)),PERCENTILE(MIN(Results.LuckyNo)),PERCENTILE(SUM(Results.LuckyNo)),PERCENTILE(NUM_UNIQUE(Results.PrizeType)),PERCENTILE(LAST(Results.TotalStrike)),PERCENTILE(STD(Results.DrawNo)),PERCENTILE(SKEW(Results.DrawNo)),"PERCENTILE(TREND(Results.LuckyNo, DrawDate))",PERCENTILE(SUM(Results.TotalStrike)),PERCENTILE(STD(Results.TotalStrike)),PERCENTILE(SKEW(Results.TotalStrike)),PERCENTILE(SUM(Results.DrawNo)),"PERCENTILE(TREND(Results.DrawNo, DrawDate))",PERCENTILE(LAST(Results.LuckyNo)),PERCENTILE(MAX(Results.LuckyNo)),TotalStrike,Label,MODE(Results.PrizeType)_Prize,LAST(Results.PrizeType)_Prize,month,year
959883,4949,2019-12-01,47194.664306,0.0,0.0,448916,1,4949,348411,1,4949,384212.750000,1,4949,1.169754,0.0,0.0,93869,448916,1,4949,5.935680e+07,4,1536851,4,19796,0.0,0.0,0.033859,1,2,20,276998340.0,7603200.0,8.363249e+07,23598.998857,5.394777e+07,0.101905,0.000000,10854.404586,...,0.0,2.901048e+06,0.032043,4998.033433,4998.033433,10,12,0.50005,0.99900,0.9942,0.50005,0.27925,0.50005,0.50005,0.50005,0.50005,0.2226,0.4950,0.27925,0.01360,0.4950,0.20680,0.50005,0.50005,0.0023,0.973776,0.50005,0.01360,0.50005,0.50005,0.0753,0.998799,0.4950,0.4950,0.0,0,1,1,12,2019
959884,3479,2019-12-01,55521.354773,0.0,0.0,500819,1,3479,350911,1,3479,390496.333333,1,3479,2.170297,0.0,0.0,105822,500819,1,3479,5.339520e+07,6,2342978,6,20874,0.0,0.0,0.033723,1,4,10,272764740.0,4233600.0,9.878460e+07,27771.803797,6.336393e+07,0.119884,0.000000,12773.786712,...,0.0,2.900992e+06,0.032043,4997.881393,4997.881393,9,25,0.50005,0.99915,0.9960,0.50005,0.93290,0.50005,0.50005,0.50005,0.50005,0.1239,0.3480,0.93290,0.07015,0.3480,0.21795,0.50005,0.50005,0.0046,0.999399,0.50005,0.07015,0.50005,0.50005,0.3051,0.997498,0.3480,0.3480,0.0,0,1,1,12,2019
959885,6512,2019-12-01,55311.481712,0.0,0.0,488518,1,6512,350911,1,6512,411514.142857,1,6512,0.002989,0.0,0.0,102989,488518,1,6512,4.062240e+07,7,2880599,7,45584,0.0,0.0,0.033912,1,4,10,272764740.0,0.0,9.786037e+07,27655.763726,6.313722e+07,0.119431,0.000000,12720.362426,...,0.0,2.900990e+06,0.032044,4998.032926,4998.032926,12,30,0.50005,0.99915,0.9991,0.50005,0.70425,0.50005,0.50005,0.50005,0.50005,0.0138,0.6513,0.70425,0.13030,0.6513,0.47660,0.50005,0.50005,0.0044,0.501051,0.50005,0.13030,0.50005,0.50005,0.5128,0.999600,0.6513,0.

In [ ]:
compare_with = compare_with[compare_with['time'] == TARGET_MONTH_YEAR]

In [22]:
len(compare_with)

10000

In [ ]:
# Sort both datafroame
feature_matrix.sort_values(by=['time', 'MAX(Results.LuckyNo)'], inplace=True)
compare_with.sort_values(by=['time', 'MAX(Results.LuckyNo)'], inplace=True)
feature_matrix = feature_matrix.reset_index(drop=True)
compare_with = compare_with.reset_index(drop=True)
feature_matrix = feature_matrix[compare_with.columns]

In [24]:
feature_matrix.describe()

,NumberId,STD(Results.DrawNo),STD(Results.TotalStrike),STD(Results.LuckyNo),MAX(Results.DrawNo),MAX(Results.TotalStrike),MAX(Results.LuckyNo),MIN(Results.DrawNo),MIN(Results.TotalStrike),MIN(Results.LuckyNo),MEAN(Results.DrawNo),MEAN(Results.TotalStrike),MEAN(Results.LuckyNo),SKEW(Results.DrawNo),SKEW(Results.TotalStrike),SKEW(Results.LuckyNo),LAST(Results.results_index),LAST(Results.DrawNo),LAST(Results.TotalStrike),LAST(Results.LuckyNo),AVG_TIME_BETWEEN(Results.DrawDate),COUNT(Results),SUM(Results.DrawNo),SUM(Results.TotalStrike),SUM(Results.LuckyNo),"TREND(Results.TotalStrike, DrawDate)","TREND(Results.LuckyNo, DrawDate)","TREND(Results.DrawNo, DrawDate)",NUM_UNIQUE(Results.PrizeType),MONTH(first_Results_time),DAY(first_Results_time),TIME_SINCE(first_Results_time),TIME_SINCE_PREVIOUS(first_Results_time),STD(Results.TIME_SINCE(DrawDate)),STD(Results.CUM_MEAN(DrawNo)),STD(Results.CUM_SUM(LuckyNo)),STD(Results.PERCENTILE(DrawNo)),STD(Results.TIME_SINCE_PREVIOUS(DrawDate)),STD(Results.CUM_SUM(TotalStrike)),STD(Results.PERCENTILE(TotalStrike)),...,CUM_MEAN(SKEW(Results.TotalStrike)),CUM_MEAN(SUM(Results.DrawNo)),"CUM_MEAN(TREND(Results.DrawNo, DrawDate))",CUM_MEAN(LAST(Results.LuckyNo)),CUM_MEAN(MAX(Results.LuckyNo)),MONTH(LAST(Results.DrawDate)),DAY(LAST(Results.DrawDate)),PERCENTILE(SKEW(Results.LuckyNo)),PERCENTILE(MIN(Results.DrawNo)),PERCENTILE(MEAN(Results.DrawNo)),PERCENTILE(STD(Results.LuckyNo)),PERCENTILE(LAST(Results.DrawNo)),PERCENTILE(MAX(Results.TotalStrike)),"PERCENTILE(TREND(Results.TotalStrike, DrawDate))",PERCENTILE(MEAN(Results.TotalStrike)),PERCENTILE(MIN(Results.TotalStrike)),PERCENTILE(AVG_TIME_BETWEEN(Results.DrawDate)),PERCENTILE(MEAN(Results.LuckyNo)),PERCENTILE(MAX(Results.DrawNo)),PERCENTILE(COUNT(Results)),PERCENTILE(MIN(Results.LuckyNo)),PERCENTILE(SUM(Results.LuckyNo)),PERCENTILE(NUM_UNIQUE(Results.PrizeType)),PERCENTILE(LAST(Results.TotalStrike)),PERCENTILE(STD(Results.DrawNo)),PERCENTILE(SKEW(Results.DrawNo)),"PERCENTILE(TREND(Results.LuckyNo, DrawDate))",PERCENTILE(SUM(Results.TotalStrike)),PERCENTILE(STD(Results.TotalStrike)),PERCENTILE(SKEW(Results.TotalStrike)),PERCENTILE(SUM(Results.DrawNo)),"PERCENTILE(TREND(Results.DrawNo, DrawDate))",PERCENTILE(LAST(Results.LuckyNo)),PERCENTILE(MAX(Results.LuckyNo)),TotalStrike,Label,MODE(Results.PrizeType)_Prize,LAST(Results.PrizeType)_Prize,month,year
count,10000.00000,10000.000000,10000.0,10000.0,10000.000000,10000.0,10000.00000,10000.000000,10000.0,10000.00000,10000.000000,10000.0,10000.00000,9991.000000,9991.0,9991.0,10000.000000,10000.000000,10000.0,10000.00000,1.000000e+04,10000.000000,1.000000e+04,10000.000000,10000.00000,9991.0,9991.0,9991.000000,10000.0,10000.000000,10000.000000,1.000000e+04,9.999000e+03,1.000000e+04,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.0,...,9991.0,1.000000e+04,9991.000000,10000.000000,10000.000000,10000.000000,10000.000000,9.991000e+03,10000.000000,10000.000000,1.000000e+04,10000.000000,1.000000e+04,9.991000e+03,1.000000e+04,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,1.000000e+04,10000.000000,9991.000000,9.991000e+03,10000.000000,1.000000e+04,9.991000e+03,10000.000000,9991.000000,10000.000000,10000.000000,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0
mean,4999.50000,131752.355861,0.0,0.0,460862.613300,1.0,4999.50000,83988.676000,1.0,4999.50000,272245.280105,1.0,4999.50000,0.002373,0.0,0.0,96621.589500,460862.613300,1.0,4999.50000,8.171873e+07,10.651300,2.900347e+06,10.651300,52983.24220,0.0,0.0,0.032070,1.0,6.671400,15.660900,7.834829e+08,7.276472e+04,2.465004e+08,65872.077932,1.507695e+08,0.284474,23396.798164,30300.182060,0.0,...,0.0,2.938868e+06,0.031869,4997.240055,4997.240055,6.766000,15.883900,5.000500e-01,0.500050,0.500050,5.000500e-01,0.500050,5.000500e-01,5.000500e-01,5.000500e-01,5.000500e-01,0.500050,0.500050,0.500050,0.500050,0.500050,0.500050,5.000500e-01,5.000500e-01,0.500050,0.500050,5.000500e-01,0.500050,5.000500e-01,

In [25]:
compare_with.describe()

,NumberId,STD(Results.DrawNo),STD(Results.TotalStrike),STD(Results.LuckyNo),MAX(Results.DrawNo),MAX(Results.TotalStrike),MAX(Results.LuckyNo),MIN(Results.DrawNo),MIN(Results.TotalStrike),MIN(Results.LuckyNo),MEAN(Results.DrawNo),MEAN(Results.TotalStrike),MEAN(Results.LuckyNo),SKEW(Results.DrawNo),SKEW(Results.TotalStrike),SKEW(Results.LuckyNo),LAST(Results.results_index),LAST(Results.DrawNo),LAST(Results.TotalStrike),LAST(Results.LuckyNo),AVG_TIME_BETWEEN(Results.DrawDate),COUNT(Results),SUM(Results.DrawNo),SUM(Results.TotalStrike),SUM(Results.LuckyNo),"TREND(Results.TotalStrike, DrawDate)","TREND(Results.LuckyNo, DrawDate)","TREND(Results.DrawNo, DrawDate)",NUM_UNIQUE(Results.PrizeType),MONTH(first_Results_time),DAY(first_Results_time),TIME_SINCE(first_Results_time),TIME_SINCE_PREVIOUS(first_Results_time),STD(Results.TIME_SINCE(DrawDate)),STD(Results.CUM_MEAN(DrawNo)),STD(Results.CUM_SUM(LuckyNo)),STD(Results.PERCENTILE(DrawNo)),STD(Results.TIME_SINCE_PREVIOUS(DrawDate)),STD(Results.CUM_SUM(TotalStrike)),STD(Results.PERCENTILE(TotalStrike)),...,CUM_MEAN(SKEW(Results.TotalStrike)),CUM_MEAN(SUM(Results.DrawNo)),"CUM_MEAN(TREND(Results.DrawNo, DrawDate))",CUM_MEAN(LAST(Results.LuckyNo)),CUM_MEAN(MAX(Results.LuckyNo)),MONTH(LAST(Results.DrawDate)),DAY(LAST(Results.DrawDate)),PERCENTILE(SKEW(Results.LuckyNo)),PERCENTILE(MIN(Results.DrawNo)),PERCENTILE(MEAN(Results.DrawNo)),PERCENTILE(STD(Results.LuckyNo)),PERCENTILE(LAST(Results.DrawNo)),PERCENTILE(MAX(Results.TotalStrike)),"PERCENTILE(TREND(Results.TotalStrike, DrawDate))",PERCENTILE(MEAN(Results.TotalStrike)),PERCENTILE(MIN(Results.TotalStrike)),PERCENTILE(AVG_TIME_BETWEEN(Results.DrawDate)),PERCENTILE(MEAN(Results.LuckyNo)),PERCENTILE(MAX(Results.DrawNo)),PERCENTILE(COUNT(Results)),PERCENTILE(MIN(Results.LuckyNo)),PERCENTILE(SUM(Results.LuckyNo)),PERCENTILE(NUM_UNIQUE(Results.PrizeType)),PERCENTILE(LAST(Results.TotalStrike)),PERCENTILE(STD(Results.DrawNo)),PERCENTILE(SKEW(Results.DrawNo)),"PERCENTILE(TREND(Results.LuckyNo, DrawDate))",PERCENTILE(SUM(Results.TotalStrike)),PERCENTILE(STD(Results.TotalStrike)),PERCENTILE(SKEW(Results.TotalStrike)),PERCENTILE(SUM(Results.DrawNo)),"PERCENTILE(TREND(Results.DrawNo, DrawDate))",PERCENTILE(LAST(Results.LuckyNo)),PERCENTILE(MAX(Results.LuckyNo)),TotalStrike,Label,MODE(Results.PrizeType)_Prize,LAST(Results.PrizeType)_Prize,month,year
count,10000.00000,10000.000000,10000.0,10000.0,10000.000000,10000.0,10000.00000,10000.000000,10000.0,10000.00000,10000.000000,10000.0,10000.00000,9991.000000,9991.0,9991.0,10000.000000,10000.000000,10000.0,10000.00000,1.000000e+04,10000.000000,1.000000e+04,10000.000000,10000.00000,9991.0,9991.0,9991.000000,10000.0,10000.000000,10000.000000,1.000000e+04,9.999000e+03,1.000000e+04,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.0,...,9991.0,1.000000e+04,9991.000000,10000.000000,10000.000000,10000.000000,10000.000000,9.991000e+03,10000.000000,10000.000000,1.000000e+04,10000.000000,1.000000e+04,9.991000e+03,1.000000e+04,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,1.000000e+04,10000.000000,9991.000000,9.991000e+03,10000.000000,1.000000e+04,9.991000e+03,10000.000000,9991.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.0,10000.0,10000.0,10000.0
mean,4999.50000,131752.355861,0.0,0.0,460862.613300,1.0,4999.50000,83988.676000,1.0,4999.50000,272245.280105,1.0,4999.50000,0.002373,0.0,0.0,96621.589500,460862.613300,1.0,4999.50000,8.171873e+07,10.651300,2.900347e+06,10.651300,52983.24220,0.0,0.0,0.032070,1.0,6.671400,15.660900,7.834829e+08,7.276472e+04,2.465004e+08,65872.077932,1.507695e+08,0.284474,23396.798164,30300.182060,0.0,...,0.0,2.938868e+06,0.031869,4997.240055,4997.240055,6.766000,15.883900,5.000500e-01,0.500050,0.500050,5.000500e-01,0.500050,5.000500e-01,5.000500e-01,5.000500e-01,5.000500e-01,0.500050,0.500050,0.500050,0.500050,0.500050,0.500050,5.000500e-01,5.000500e-01,0.500050,0.500050,5.000500e-01,0.500050,5.0

In [26]:
# compare these 2 dataframe 
feature_matrix.tail(10)

,NumberId,time,STD(Results.DrawNo),STD(Results.TotalStrike),STD(Results.LuckyNo),MAX(Results.DrawNo),MAX(Results.TotalStrike),MAX(Results.LuckyNo),MIN(Results.DrawNo),MIN(Results.TotalStrike),MIN(Results.LuckyNo),MEAN(Results.DrawNo),MEAN(Results.TotalStrike),MEAN(Results.LuckyNo),SKEW(Results.DrawNo),SKEW(Results.TotalStrike),SKEW(Results.LuckyNo),LAST(Results.results_index),LAST(Results.DrawNo),LAST(Results.TotalStrike),LAST(Results.LuckyNo),AVG_TIME_BETWEEN(Results.DrawDate),COUNT(Results),SUM(Results.DrawNo),SUM(Results.TotalStrike),SUM(Results.LuckyNo),"TREND(Results.TotalStrike, DrawDate)","TREND(Results.LuckyNo, DrawDate)","TREND(Results.DrawNo, DrawDate)",NUM_UNIQUE(Results.PrizeType),MONTH(first_Results_time),DAY(first_Results_time),TIME_SINCE(first_Results_time),TIME_SINCE_PREVIOUS(first_Results_time),STD(Results.TIME_SINCE(DrawDate)),STD(Results.CUM_MEAN(DrawNo)),STD(Results.CUM_SUM(LuckyNo)),STD(Results.PERCENTILE(DrawNo)),STD(Results.TIME_SINCE_PREVIOUS(DrawDate)),STD(Results.CUM_SUM(TotalStrike)),...,CUM_MEAN(SKEW(Results.TotalStrike)),CUM_MEAN(SUM(Results.DrawNo)),"CUM_MEAN(TREND(Results.DrawNo, DrawDate))",CUM_MEAN(LAST(Results.LuckyNo)),CUM_MEAN(MAX(Results.LuckyNo)),MONTH(LAST(Results.DrawDate)),DAY(LAST(Results.DrawDate)),PERCENTILE(SKEW(Results.LuckyNo)),PERCENTILE(MIN(Results.DrawNo)),PERCENTILE(MEAN(Results.DrawNo)),PERCENTILE(STD(Results.LuckyNo)),PERCENTILE(LAST(Results.DrawNo)),PERCENTILE(MAX(Results.TotalStrike)),"PERCENTILE(TREND(Results.TotalStrike, DrawDate))",PERCENTILE(MEAN(Results.TotalStrike)),PERCENTILE(MIN(Results.TotalStrike)),PERCENTILE(AVG_TIME_BETWEEN(Results.DrawDate)),PERCENTILE(MEAN(Results.LuckyNo)),PERCENTILE(MAX(Results.DrawNo)),PERCENTILE(COUNT(Results)),PERCENTILE(MIN(Results.LuckyNo)),PERCENTILE(SUM(Results.LuckyNo)),PERCENTILE(NUM_UNIQUE(Results.PrizeType)),PERCENTILE(LAST(Results.TotalStrike)),PERCENTILE(STD(Results.DrawNo)),PERCENTILE(SKEW(Results.DrawNo)),"PERCENTILE(TREND(Results.LuckyNo, DrawDate))",PERCENTILE(SUM(Results.TotalStrike)),PERCENTILE(STD(Results.TotalStrike)),PERCENTILE(SKEW(Results.TotalStrike)),PERCENTILE(SUM(Results.DrawNo)),"PERCENTILE(TREND(Results.DrawNo, DrawDate))",PERCENTILE(LAST(Results.LuckyNo)),PERCENTILE(MAX(Results.LuckyNo)),TotalStrike,Label,MODE(Results.PrizeType)_Prize,LAST(Results.PrizeType)_Prize,month,year
9990,9990,2019-12-01,158865.967466,0.0,0.0,498619,1,9990,74194,1,9990,241867.727273,1,9990,0.581222,0.0,0.0,105310,498619,1,9990,7.930656e+07,11,2660545,11,109890,0.0,0.0,0.032093,1,6,23,802828740.0,0.0,2.969490e+08,79431.803264,1.817704e+08,0.343029,78151.740515,36537.767743,...,0.0,2.914140e+06,0.031865,5008.984404,5008.984404,8,10,0.50005,0.53860,0.2367,0.50005,0.88680,0.50005,0.50005,0.50005,0.50005,0.58225,0.9991,0.88680,0.56205,0.9991,0.9274,0.50005,0.50005,0.8988,0.854269,0.50005,0.56205,0.50005,0.50005,0.4261,0.540587,0.9991,0.9991,0.0,0,1,1,12,2019
9991,9991,2019-12-01,158803.451902,0.0,0.0,484518,1,9991,40992,1,9991,302801.000000,1,9991,-0.539629,0.0,0.0,102051,484518,1,9991,5.954811e+07,15,4542015,15,149865,0.0,0.0,0.031981,1,5,10,869702340.0,0.0,2.978367e+08,79398.220435,1.817595e+08,0.342894,66925.152222,36521.925601,...,0.0,3.175738e+06,0.031856,5116.724138,5116.724138,10,10,0.50005,0.00580,0.7656,0.50005,0.64075,0.50005,0.50005,0.50005,0.50005,0.22770,0.9992,0.64075,0.90180,0.9992,0.9907,0.50005,0.50005,0.8982,0.158042,0.50005,0.90180,0.50005,0.50005,0.9416,0.401661,0.9992,0.9992,0.0,0,1,1,12,2019
9992,9992,2019-12-01,162256.185223,0.0,0.0,492619,1,9992,48792,1,9992,307577.900000,1,9992,-0.462118,0.0,0.0,103919,492619,1,9992,9.252480e+07,10,3075779,10,99920,0.0,0.0,0.032120,1,11,8,853977540.0,0.0,3.030231e+08,81129.643219,1.856522e+08,0.350349,0.000000,37318.610733,...,0.0,2.949088e+06,0.031798,4987.151355,4987.151355,3,30,0.50005,0.17050,0.7982,0.50005,0.77340,0.50005,0.50005,0.50005,0.50005,0.74080,0.9993,0.77340,0.44340,0.9993,0.8912,0.50005,0.50005,0.9218,0.194575,0.50005,0.44340,0.50005,0.50005,0.5970,0.572715,0.9993,0.

In [27]:
compare_with.tail(10)

,NumberId,time,STD(Results.DrawNo),STD(Results.TotalStrike),STD(Results.LuckyNo),MAX(Results.DrawNo),MAX(Results.TotalStrike),MAX(Results.LuckyNo),MIN(Results.DrawNo),MIN(Results.TotalStrike),MIN(Results.LuckyNo),MEAN(Results.DrawNo),MEAN(Results.TotalStrike),MEAN(Results.LuckyNo),SKEW(Results.DrawNo),SKEW(Results.TotalStrike),SKEW(Results.LuckyNo),LAST(Results.results_index),LAST(Results.DrawNo),LAST(Results.TotalStrike),LAST(Results.LuckyNo),AVG_TIME_BETWEEN(Results.DrawDate),COUNT(Results),SUM(Results.DrawNo),SUM(Results.TotalStrike),SUM(Results.LuckyNo),"TREND(Results.TotalStrike, DrawDate)","TREND(Results.LuckyNo, DrawDate)","TREND(Results.DrawNo, DrawDate)",NUM_UNIQUE(Results.PrizeType),MONTH(first_Results_time),DAY(first_Results_time),TIME_SINCE(first_Results_time),TIME_SINCE_PREVIOUS(first_Results_time),STD(Results.TIME_SINCE(DrawDate)),STD(Results.CUM_MEAN(DrawNo)),STD(Results.CUM_SUM(LuckyNo)),STD(Results.PERCENTILE(DrawNo)),STD(Results.TIME_SINCE_PREVIOUS(DrawDate)),STD(Results.CUM_SUM(TotalStrike)),...,CUM_MEAN(SKEW(Results.TotalStrike)),CUM_MEAN(SUM(Results.DrawNo)),"CUM_MEAN(TREND(Results.DrawNo, DrawDate))",CUM_MEAN(LAST(Results.LuckyNo)),CUM_MEAN(MAX(Results.LuckyNo)),MONTH(LAST(Results.DrawDate)),DAY(LAST(Results.DrawDate)),PERCENTILE(SKEW(Results.LuckyNo)),PERCENTILE(MIN(Results.DrawNo)),PERCENTILE(MEAN(Results.DrawNo)),PERCENTILE(STD(Results.LuckyNo)),PERCENTILE(LAST(Results.DrawNo)),PERCENTILE(MAX(Results.TotalStrike)),"PERCENTILE(TREND(Results.TotalStrike, DrawDate))",PERCENTILE(MEAN(Results.TotalStrike)),PERCENTILE(MIN(Results.TotalStrike)),PERCENTILE(AVG_TIME_BETWEEN(Results.DrawDate)),PERCENTILE(MEAN(Results.LuckyNo)),PERCENTILE(MAX(Results.DrawNo)),PERCENTILE(COUNT(Results)),PERCENTILE(MIN(Results.LuckyNo)),PERCENTILE(SUM(Results.LuckyNo)),PERCENTILE(NUM_UNIQUE(Results.PrizeType)),PERCENTILE(LAST(Results.TotalStrike)),PERCENTILE(STD(Results.DrawNo)),PERCENTILE(SKEW(Results.DrawNo)),"PERCENTILE(TREND(Results.LuckyNo, DrawDate))",PERCENTILE(SUM(Results.TotalStrike)),PERCENTILE(STD(Results.TotalStrike)),PERCENTILE(SKEW(Results.TotalStrike)),PERCENTILE(SUM(Results.DrawNo)),"PERCENTILE(TREND(Results.DrawNo, DrawDate))",PERCENTILE(LAST(Results.LuckyNo)),PERCENTILE(MAX(Results.LuckyNo)),TotalStrike,Label,MODE(Results.PrizeType)_Prize,LAST(Results.PrizeType)_Prize,month,year
9990,9990,2019-12-01,158865.967466,0.0,0.0,498619,1,9990,74194,1,9990,241867.727273,1,9990,0.581222,0.0,0.0,105310,498619,1,9990,7.930656e+07,11,2660545,11,109890,0.0,0.0,0.032093,1,6,23,802828740.0,0.0,2.969490e+08,79431.803264,1.817704e+08,0.343029,78151.740515,36537.767743,...,0.0,2.914140e+06,0.031865,5008.984404,5008.984404,8,10,0.50005,0.53860,0.2367,0.50005,0.88680,0.50005,0.50005,0.50005,0.50005,0.58225,0.9991,0.88680,0.56205,0.9991,0.9274,0.50005,0.50005,0.8988,0.854269,0.50005,0.56205,0.50005,0.50005,0.4261,0.540587,0.9991,0.9991,1.0,1,1,1,12,2019
9991,9991,2019-12-01,158803.451902,0.0,0.0,484518,1,9991,40992,1,9991,302801.000000,1,9991,-0.539629,0.0,0.0,102051,484518,1,9991,5.954811e+07,15,4542015,15,149865,0.0,0.0,0.031981,1,5,10,869702340.0,0.0,2.978367e+08,79398.220435,1.817595e+08,0.342894,66925.152222,36521.925601,...,0.0,3.175738e+06,0.031856,5116.724138,5116.724138,10,10,0.50005,0.00580,0.7656,0.50005,0.64075,0.50005,0.50005,0.50005,0.50005,0.22770,0.9992,0.64075,0.90180,0.9992,0.9907,0.50005,0.50005,0.8982,0.158042,0.50005,0.90180,0.50005,0.50005,0.9416,0.401661,0.9992,0.9992,0.0,0,1,1,12,2019
9992,9992,2019-12-01,162256.185223,0.0,0.0,492619,1,9992,48792,1,9992,307577.900000,1,9992,-0.462118,0.0,0.0,103919,492619,1,9992,9.252480e+07,10,3075779,10,99920,0.0,0.0,0.032120,1,11,8,853977540.0,0.0,3.030231e+08,81129.643219,1.856522e+08,0.350349,0.000000,37318.610733,...,0.0,2.949088e+06,0.031798,4987.151355,4987.151355,3,30,0.50005,0.17050,0.7982,0.50005,0.77340,0.50005,0.50005,0.50005,0.50005,0.74080,0.9993,0.77340,0.44340,0.9993,0.8912,0.50005,0.50005,0.9218,0.194575,0.50005,0.44340,0.50005,0.50005,0.5970,0.572715,0.9993,0.

In [29]:
feature_matrix['STD(Results.TotalStrike)'].equals(compare_with['STD(Results.TotalStrike)'])

False

In [31]:
for col in feature_matrix.columns:
  is_equals = feature_matrix[col].equals(compare_with[col])
  if not is_equals:
    print(col, is_equals)

TotalStrike False
Label False
